# Introduction to Bedrock - Fine-Tuning

> *If you see errors, you may need to be allow-listed for the Bedrock models used by this notebook*

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*


In this demo notebook, we demonstrate how to use the Bedrock Python SDK for fine-tuning Bedrock models with your own data. If you have text samples to train and want to adapt the Bedrock models to your domain, you can further fine-tune the Bedrock foundation models by providing your own training datasets. You can upload your datasets to Amazon S3, and provide the S3 bucket path while configuring a Bedrock fine-tuning job. You can also adjust hyper parameters (learning rate, epoch, and batch size) for fine-tuning. After the fine-tuning job of the model with your dataset has completed, you can start using the model for inference in the Bedrock playground application. You can select the fine-tuned model and submit a prompt to the fine-tuned model along with a set of model parameters. The fine-tuned model should generate texts to be more alike your text samples. 

-----------

1. Setup
2. Fine-tuning
3. Testing the fine-tuned model

 Note: This notebook was tested in Amazon SageMaker Studio with Python 3 (Data Science 2.0) kernel.

---

## 1. Setup

In [2]:
%pip install -U --force-reinstall pandas==2.1.2 datasets==2.14.6

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for pandas==2.1.2 from https://files.pythonhosted.org/packages/02/52/815f643ed3afb3365354548b3c8b557dbf926a65c40ad5b6d9e455147c7e/pandas-2.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for datasets==2.14.6 from https://files.pythonhosted.org/packages/7c/55/b3432f43d6d7fee999bb23a547820d74c48ec540f5f7842e41aa5d8d5f3a/datasets-2.14.6-py3-none-any.whl.metadata
  Obtaining dependency information for numpy<2,>=1.22.4 from https://files.pythonhosted.org/packages/64/41/284783f1014685201e447ea976e85fed0e351f5debbaf3ee6d7645521f1d/numpy-1.26.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 255.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.7/247.7 kB 176.5 MB/s eta 0:00:00
  Obtaining dependency information for pytz>=2020.1 from htt

In [3]:
# %pip install -U boto3-1.28.60-py3-none-any.whl \
#                botocore-1.31.60-py3-none-any.whl \
#                awscli-1.29.60-py3-none-any.whl \
#                --force-reinstall --quiet

In [4]:
%pip list | grep boto3

boto3                                1.29.6

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


#### Now let's set up our connection to the Amazon Bedrock SDK using Boto3

In [5]:
#### Un comment the following lines to run from your local environment outside of the AWS account with Bedrock access

#import os
#os.environ['BEDROCK_ASSUME_ROLE'] = '<YOUR_VALUES>'
#os.environ['AWS_PROFILE'] = '<YOUR_VALUES>'

In [6]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

In [7]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'b0d9b327-9fa2-4162-a090-873b334e3882',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 27 Nov 2023 03:57:55 GMT',
   'content-type': 'application/json',
   'content-length': '9628',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b0d9b327-9fa2-4162-a090-873b334e3882'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND']},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-medium',
   'modelId': 'amazon.titan-tg1-medium',
   'modelName': 'Titan Text Medium',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'r

In [8]:
import sagemaker

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()
role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


### Invoke Model before Fine-Training

In [9]:
command_base_model_id = "cohere.command-light-text-v14"

In [10]:
base_model_id = "cohere.command-light-text-v14:7:4k"

### Convert the dataset into jsonlines format

In [11]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="gaia_questions_answers.csv")

In [12]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 110
    })
})


In [13]:
def wrap_instruction_fn(example):
    prompt = 'Answer the following question:\n\n'
    end_prompt = '\n\nAnswer: '
    example["instruction"] = prompt + example["question"] + end_prompt
    return example

In [14]:
dataset['train']\
  .filter(lambda example: example['question'] and example['answer'])\
  .select_columns(['question', 'answer'])\
  .map(wrap_instruction_fn)\
  .rename_column('instruction', 'prompt')\
  .rename_column('answer', 'completion')\
  .remove_columns(['question'])\
  .to_json('./train-question-answer.jsonl', index=False)

# dataset['validation']\
#   .filter(lambda example: example['question'])\
#   .select_columns(['question', 'answer'])\
#   .map(wrap_instruction_fn)\
#   .rename_column('instruction', 'input')\
#   .rename_column('answer', 'output')\
#   .to_json('./validation-summarization.jsonl', index=False)

# dataset['test']\
#   .filter(lambda example: example['question'])\
#   .select_columns(['question', 'answer'])\
#   .map(wrap_instruction_fn)\
#   .rename_column('instruction', 'input')\
#   .rename_column('answer', 'output')\
#   .to_json('./test-summarization.jsonl', index=False)

#  .remove_columns(['Unnamed: 0.1', 'Unnamed: 0', 'question_id', 'title', 'answer_id', 'expertreviewed', 'upvotedcount', 'tags'])\

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

42498

In [15]:
import pandas as pd
df = pd.read_json("./train-question-answer.jsonl", lines=True)
df

,completion,prompt
0,"Intelligent search, automated customer-support...",Answer the following question:\n\nWhat are som...
1,Foundation models are large and complex neural...,Answer the following question:\n\nHow are foun...
2,"The generative AI project life cycle, though n...",Answer the following question:\n\nCan you desc...
3,AWS offers a range of frameworks and infrastru...,Answer the following question:\n\nWhat makes A...
4,"AWS offers increased flexibility, choice, ente...",Answer the following question:\n\nHow does Gen...
...,...,...
105,Textual inversion is a lightweight fine-tuning...,Answer the following question:\n\nWhat is text...
106,Human Alignment with Reinforcement Learning fr...,Answer the following question:\n\nHow does hum...
107,PEFT-LoRA (Parameter-Efficient Fine-Tuning wit...,Answer the following question:\n\nHow do PEFT-...
108,Fine-tuning Stable Diffusion models allows cus...,Answer the following question:\n\nWhat are the...


In [16]:
data = "./train-question-answer.jsonl"

Read the JSON line file into an object like any normal file

In [17]:
with open(data) as f:
    lines = f.read().splitlines()

#### Load the ‘lines’ object into a pandas Data Frame.

In [18]:
import pandas as pd
df_inter = pd.DataFrame(lines)
df_inter.columns = ['json_element']

This intermediate data frame will have only one column with each json object in a row. A sample output is given below.

In [19]:
df_inter['json_element'].apply(json.loads)

0      {'completion': 'Intelligent search, automated ...
1      {'completion': 'Foundation models are large an...
2      {'completion': 'The generative AI project life...
3      {'completion': 'AWS offers a range of framewor...
4      {'completion': 'AWS offers increased flexibili...
                             ...                        
105    {'completion': 'Textual inversion is a lightwe...
106    {'completion': 'Human Alignment with Reinforce...
107    {'completion': 'PEFT-LoRA (Parameter-Efficient...
108    {'completion': 'Fine-tuning Stable Diffusion m...
109    {'completion': 'Fine-tuning techniques for dif...
Name: json_element, Length: 110, dtype: object

Now we will apply json loads function on each row of the ‘json_element’ column. ‘json.loads’ is a decoder function in python which is used to decode a json object into a dictionary. ‘apply’ is a popular function in pandas that takes any function and applies to each row of the pandas dataframe or series.

In [20]:
df_final = pd.json_normalize(df_inter['json_element'].apply(json.loads))

Once decoding is done we will apply the json normalize function to the above result. json normalize will convert any semi-structured json data into a flat table. Here it converts the JSON ‘keys’ to columns and its corresponding values to row elements.

In [21]:
df_final

,completion,prompt
0,"Intelligent search, automated customer-support...",Answer the following question:\n\nWhat are som...
1,Foundation models are large and complex neural...,Answer the following question:\n\nHow are foun...
2,"The generative AI project life cycle, though n...",Answer the following question:\n\nCan you desc...
3,AWS offers a range of frameworks and infrastru...,Answer the following question:\n\nWhat makes A...
4,"AWS offers increased flexibility, choice, ente...",Answer the following question:\n\nHow does Gen...
...,...,...
105,Textual inversion is a lightweight fine-tuning...,Answer the following question:\n\nWhat is text...
106,Human Alignment with Reinforcement Learning fr...,Answer the following question:\n\nHow does hum...
107,PEFT-LoRA (Parameter-Efficient Fine-Tuning wit...,Answer the following question:\n\nHow do PEFT-...
108,Fine-tuning Stable Diffusion models allows cus...,Answer the following question:\n\nWhat are the...


### Uploading data to S3

Next, we need to upload our training dataset to S3:

In [22]:
s3_location = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/train-question-answer.jsonl"
s3_output = f"s3://{sagemaker_session_bucket}/bedrock/finetuning/output"

In [23]:
!aws s3 cp ./train-question-answer.jsonl $s3_location

upload: ./train-question-answer.jsonl to s3://sagemaker-us-east-1-079002598131/bedrock/finetuning/train-question-answer.jsonl


Now we can create the fine-tuning job. 

### ^^ **Note:** Make sure the IAM role you're using has these [IAM policies](https://docs.aws.amazon.com/bedrock/latest/userguide/model-customization-iam-role.html) attached that allow Amazon Bedrock access to the specified S3 buckets ^^

## 2. Fine-tuning

In [24]:
import time
timestamp = int(time.time())

In [25]:
job_name = "cohere-{}".format(timestamp)
job_name

'cohere-1701057480'

In [26]:
custom_model_name = "custom-{}".format(job_name)
custom_model_name

'custom-cohere-1701057480'

In [27]:
bedrock.create_model_customization_job(
    jobName=job_name,
    customModelName=custom_model_name,
    roleArn=role,
    baseModelIdentifier=base_model_id,
    hyperParameters = {
        "epochCount": "10",
        "batchSize": "8",
        "learningRate": "0.000005",
    },
    trainingDataConfig={"s3Uri": s3_location},
    outputDataConfig={"s3Uri": s3_output},
)

{'ResponseMetadata': {'RequestId': '3817d5eb-6fd5-4b4c-91cd-c2cfc4399d3b',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 27 Nov 2023 03:58:00 GMT',
   'content-type': 'application/json',
   'content-length': '123',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3817d5eb-6fd5-4b4c-91cd-c2cfc4399d3b'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:079002598131:model-customization-job/cohere.command-light-text-v14:7:4k/ipm2vviqeqvn'}

In [28]:
status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
status

'InProgress'

# Let's periodically check in on the progress.
### The next cell might run for ~40min

In [29]:
import time

status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]

while status == "InProgress":
    print(status)
    time.sleep(30)
    status = bedrock.get_model_customization_job(jobIdentifier=job_name)["status"]
    
print(status)

InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Failed


In [30]:
completed_job = bedrock.get_model_customization_job(jobIdentifier=job_name)
completed_job

{'ResponseMetadata': {'RequestId': 'd14aaf5d-3e23-4a7c-8f6c-1949446c809d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 27 Nov 2023 04:27:42 GMT',
   'content-type': 'application/json',
   'content-length': '1331',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd14aaf5d-3e23-4a7c-8f6c-1949446c809d'},
  'RetryAttempts': 0},
 'jobArn': 'arn:aws:bedrock:us-east-1:079002598131:model-customization-job/cohere.command-light-text-v14:7:4k/ipm2vviqeqvn',
 'jobName': 'cohere-1701057480',
 'outputModelName': 'custom-cohere-1701057480',
 'clientRequestToken': '0279c513-653c-4001-a55d-a1436266a835',
 'roleArn': 'arn:aws:iam::079002598131:role/service-role/AmazonSageMaker-ExecutionRole-20220804T150518',
 'status': 'Failed',
 'failureMessage': 'Validation error: After preprocessing, duplicates and large examples were removed.At least 32 examples needed if passing only train file',
 'creationTime': datetime.datetime(2023, 11, 27, 3, 58, 0, 853000, tzinfo=tzlocal()),
 'lastModifiedT

## 3. Testing

Now we can test the fine-tuned model

In [31]:
bedrock.list_custom_models()

{'ResponseMetadata': {'RequestId': '921684a8-da34-41e0-b7b0-0295249070d0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 27 Nov 2023 04:27:42 GMT',
   'content-type': 'application/json',
   'content-length': '1352',
   'connection': 'keep-alive',
   'x-amzn-requestid': '921684a8-da34-41e0-b7b0-0295249070d0'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1:079002598131:custom-model/meta.llama2-13b-v1:0:4k/090h29lu0qu6',
   'modelName': 'custom-llama2-1701035879',
   'creationTime': datetime.datetime(2023, 11, 26, 21, 57, 59, 827000, tzinfo=tzlocal()),
   'baseModelArn': 'arn:aws:bedrock:us-east-1::foundation-model/meta.llama2-13b-v1:0:4k',
   'baseModelName': ''},
  {'modelArn': 'arn:aws:bedrock:us-east-1:079002598131:custom-model/meta.llama2-13b-v1:0:4k/ylbb90zehwl3',
   'modelName': 'custom-llama2-1701028200',
   'creationTime': datetime.datetime(2023, 11, 26, 19, 50, 1, 477000, tzinfo=tzlocal()),
   'baseModelArn': 'arn:aws:bedrock:us-ea

In [32]:
for job in bedrock.list_model_customization_jobs()["modelCustomizationJobSummaries"]:
    print("-----\n" + "jobArn: " + job["jobArn"] + "\njobName: " + job["jobName"] + "\nstatus: " + job["status"] + "\ncustomModelName: " + job["customModelName"])

-----
jobArn: arn:aws:bedrock:us-east-1:079002598131:model-customization-job/amazon.titan-text-express-v1:0:8k/g7j3engjpdlp
jobName: titan-1701058871
status: InProgress
customModelName: custom-titan-1701058871
-----
jobArn: arn:aws:bedrock:us-east-1:079002598131:model-customization-job/cohere.command-light-text-v14:7:4k/ipm2vviqeqvn
jobName: cohere-1701057480
status: Failed
customModelName: custom-cohere-1701057480
-----
jobArn: arn:aws:bedrock:us-east-1:079002598131:model-customization-job/meta.llama2-13b-v1:0:4k/nn382igms8q1
jobName: llama2-1701035879
status: Completed
customModelName: custom-llama2-1701035879
-----
jobArn: arn:aws:bedrock:us-east-1:079002598131:model-customization-job/cohere.command-light-text-v14:7:4k/v8qgvki9ae1y
jobName: cohere-1701034305
status: Failed
customModelName: custom-cohere-1701034305
-----
jobArn: arn:aws:bedrock:us-east-1:079002598131:model-customization-job/meta.llama2-13b-v1:0:4k/4nad2l4z3gq0
jobName: llama2-1701028200
status: Completed
customModelN

## GetCustomModel

In [33]:
bedrock.get_custom_model(modelIdentifier=custom_model_name)

ValidationException: An error occurred (ValidationException) when calling the GetCustomModel operation: The provided model identifier is invalid.

In [ ]:
custom_model_arn = bedrock.get_custom_model(modelIdentifier=custom_model_name)['modelArn']
custom_model_arn

In [ ]:
base_model_arn = bedrock.get_custom_model(modelIdentifier=custom_model_name)['baseModelArn']
base_model_arn

## **Note:** To invoke custom models, you need to first create a provisioned throughput resource and make requests using that resource.

In [ ]:
provisioned_model_name = "{}-provisioned".format(custom_model_name)
provisioned_model_name

## !! **Note:** SDK currently only supports 1 month and 6 months commitment terms. Go to Bedrock console to manually purchase no commitment term option for testing !!

In [ ]:
# bedrock.create_provisioned_model_throughput(
#     modelUnits = 1,
#     commitmentDuration = "OneMonth", ## Note: SDK is currently missing No Commitment option
#     provisionedModelName = provisioned_model_name,
#     modelId = base_model_arn
# ) 

## ListProvisionedModelThroughputs

In [ ]:
bedrock.list_provisioned_model_throughputs()["provisionedModelSummaries"]

## GetProvisionedModelThroughput

In [ ]:
#provisioned_model_name = "<YOUR_PROVISIONED_MODEL_NAME>" # e.g. custom-titan-1698257909-provisioned
#provisioned_model_name = "custom-titan-1700411048-provisioned"

In [ ]:
provisioned_model_arn = bedrock.get_provisioned_model_throughput(
     provisionedModelId=provisioned_model_name)["provisionedModelArn"]
provisioned_model_arn

In [ ]:
deployment_status = bedrock.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]
deployment_status

## The next cell might run for ~10min

In [ ]:
import time

deployment_status = bedrock.get_provisioned_model_throughput(
    provisionedModelId=provisioned_model_name)["status"]

while deployment_status == "Creating":
    
    print(deployment_status)
    time.sleep(30)
    deployment_status = bedrock.get_provisioned_model_throughput(
        provisionedModelId=provisioned_model_name)["status"]  
    
print(deployment_status)

# Qualitative Results with Zero Shot Inference BEFORE and AFTER Fine-Tuning

As with many GenAI applications, a qualitative approach where you ask yourself the question "is my model behaving the way it is supposed to?" is usually a good starting point. In the example below (the same one we started this notebook with), you can see how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

## Before Fine-Tuning

In [ ]:
command_base_model_id = "cohere.command-light-text-v14"

In [ ]:
prompt = "Answer the following question:\n\nHow can prompt engineering go wrong?\n\nAnswer: "

In [ ]:
body = {
    "prompt": prompt,
    "temperature": 0.5,
    "top_p": 0.9,
    "max_gen_len": 512,
}

response = bedrock_runtime.invoke_model(
    modelId=command_base_model_id, 
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

## After Fine-Tuning

In [ ]:
body = {
    "prompt": prompt,
    "temperature": 0.5,
    "top_p": 0.9,
    "max_gen_len": 512,
}

response = bedrock_runtime.invoke_model(
    modelId=provisioned_model_arn,
    body=json.dumps(body)
)

response_body = response["body"].read().decode('utf8')
print(json.loads(response_body)["generation"])

## Delete Provisioned Throughput

When you're done testing, you can delete Provisioned Throughput to stop charges

In [ ]:
# bedrock.delete_provisioned_model_throughput(
#     provisionedModelId = provisioned_model_name
# )